# 汽车投诉信息采集：
数据源：http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml
投诉编号，投诉品牌，投诉车系，投诉车型，问题简述，典型问题，投诉时间，投诉状态


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [14]:

def get_page_content(request_url):
    header = {'user-agent': "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Mobile Safari/537.36"}
    html = requests.get(request_url, headers = header, timeout = 10)
    content = html.text 
    
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup
    

In [42]:
def analysis_soup(soup):
    temp = soup.find('div', class_ = 'tslb_b')
    column_list = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status']
    df = pd.DataFrame(columns = column_list)
    tr_list = temp.find_all('tr')
    for tr in tr_list:
        td_list = tr.find_all('td')
        if len(td_list) > 0:
            #car_id, brand, car_model, car_type, car_desc, car_problem, car_datetime, car_status = [x.text for x in td_list]
            car_info_list = [x.text for x in td_list]
            temp = {}
            for i in range(len(column_list)):
                temp[column_list[i]] = car_info_list[i]
            df = df.append(temp, ignore_index=True)
    return df
                

In [43]:
page_num = 20
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'

result = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
for i in range(page_num):
    request_url = base_url+str(i+1)+'.shtml'
    soup = get_page_content(request_url)
    df = analysis_soup(soup)
    result = result.append(df)

result.to_csv('car_complain.csv', index=False)

/Users/zechaoli/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
